# LLM_ESM2_Features_Regression

As mentioned in [] notebook, protein embeddings from LLMs are useful as engineered features for a variety of downstream machine learning tasks. Here we will load an experiment dataset containing protein sequences and corresponding measured fluorescence values. We can generate ESM2 embeddings, at scale, as features in order to create a regression to predict observed fluorescence.

In [ ]:
from helpers import api_caller

TOKEN = ''

In [2]:
# Initiliaze pandarallel first so we can multiprocess DF
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True, nb_workers=12)

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [11]:
import requests, os, json, datetime

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import xgboost as xgb

from glob import glob
from sklearn import model_selection, preprocessing

sns.set_style('white')

In [12]:
os.getcwd()

'/Users/nikhil/Developer/tutorials-and-guides/protein/regression'

In [13]:
# Load sequences and fluorescence data
df = pd.concat([
    pd.read_csv(f) for f in
    glob(os.path.join('..', 'data', '*.csv'))
])

df.drop_duplicates('seq', inplace=True)

df.sample(8)

,seq,label
15015,SKGEELFSGVVPILAELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,3.616179
24482,SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDAAYGKLTLKFI...,3.463095
8241,SKGVELLTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,3.751024
3916,SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,3.258257
2179,SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,3.558424
17679,SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,1.340580
10339,SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,3.825261
18570,SKGEELFTGVVPILVELDGDVNGHKFSVPGEGEGDATYGKLTLKFI...,1.586859


Let's write a function that takes a sequence and requests its embeddings via REST API. The [ESM2 transform endpoint documentation](https://api.biolm.ai/#ae27a66e-6e2e-4b1f-a540-f426e141d16f) provides examples of the structure of a response.

In [14]:
# First let's write a function to get our API token using username/password

def get_api_token():
    """Get a BioLM API token to use with future API requests.
    
    Copied from https://api.biolm.ai/#d7f87dfd-321f-45ae-99b6-eb203519ddeb.
    """
    url = "https://biolm.ai/api/auth/token/"

    payload = json.dumps({
      "username": os.environ.get("BIOLM_USER"),
      "password": os.environ.get("BIOLM_PASS")
    })
    headers = {
      'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    return response.json()

In [15]:
api_tok = get_api_token()

os.environ['BIOLM_ACCESS'] = api_tok['access']
os.environ['BIOLM_REFRESH'] = api_tok['refresh']

Great, now we have our `access` and `refresh` tokens.

In [16]:
# Now the function to POST sequences to GPU-backed API for ESM2 tokenization

def get_embeddings(seq):
    """Make a POST request to get the ESM2 sequence embeddings for a protein.
    
    Also record the clock time to get the embeddings.
    """
    start = datetime.datetime.now()

    url = "https://biolm.ai/api/v1/models/esm2_t33_650M_UR50D/transform/"
    
    # Normally would POST multiple sequences at once for greater efficiency,
    # but for simplicity sake will do one at at time right now
    payload = json.dumps({
      "instances": [
        {
          "data": {
            "text": seq
          }
        }
      ]
    })
    
    try:
        access = os.environ.get('BIOLM_ACCESS')
        assert access
        refresh = os.environ.get('BIOLM_REFRESH')
        assert refresh
    except AssertionError:
        raise AssertionError("BioLM access or refresh token not set")
    
    headers = {
      'Cookie': 'access={};refresh={}'.format(access, refresh),
      'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    
    end = datetime.datetime.now()
    clocktime = end - start

    resp_json = response.json()
    transformations = resp_json['predictions']  # List, containing dicts for each sequence POSTed
    # List of dict_keys(['name', 'mean_representations', 'contacts', 'logits', 'attentions'])
    
    return [posted_sequence['mean_representations'] for posted_sequence in transformations]

In [17]:
test_protein = df.sample(1).seq.iloc[0]

print("Sequence length: {}\n{}".format(len(test_protein), test_protein))

Sequence length: 237
SKGEELFAGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKLEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK


We can POST that sequence

In [18]:
r = get_embeddings(test_protein)

r

[{'33': [-0.01431241724640131,
   -0.010068249888718128,
   -0.044924311339855194,
   -0.017040755599737167,
   -0.09082041680812836,
   -0.024620071053504944,
   -0.05334519222378731,
   0.016817402094602585,
   0.06651005893945694,
   -0.08746325969696045,
   0.028786756098270416,
   -0.010175399482250214,
   0.04353352636098862,
   0.14612628519535065,
   -0.004662312567234039,
   0.0038508675061166286,
   -0.015029172413051128,
   -0.012473802082240582,
   -0.0013126978883519769,
   0.015082133002579212,
   -0.007675808854401112,
   0.03126747906208038,
   -0.015358014032244682,
   0.10624054819345474,
   -0.020585287362337112,
   0.0052359336987137794,
   0.00865401141345501,
   0.01769261807203293,
   0.011765838600695133,
   -0.1671944409608841,
   0.018612895160913467,
   0.0014590698992833495,
   0.026348309591412544,
   -0.03727123513817787,
   -0.011514080688357353,
   -0.006600270513445139,
   -0.03916453570127487,
   0.036856215447187424,
   -0.03993227332830429,
   -0.058

We get back a list of dicts. Each dictionary contains the mean representations of a layer(s) from ESM2. In this case, we return the embeddings from the the final hidden layer, `33`.

Let's load this representation and look at its shape.

In [19]:
first_posted_sequence = r[0]

embed_df = pd.DataFrame([first_posted_sequence['33']])

embed_df

,0,1,2,3,4,5,6,7,8,9,...,1270,1271,1272,1273,1274,1275,1276,1277,1278,1279
0,-0.014312,-0.010068,-0.044924,-0.017041,-0.09082,-0.02462,-0.053345,0.016817,0.06651,-0.087463,...,0.138999,0.049933,-0.00549,0.082272,-0.006908,0.105805,0.091021,0.030206,-0.013018,-0.021144


We can see that while the original sequence is `237` residues, the NN uses a vector of `1280` to represent sequences. So, anytime we request an embedding for a sequence, we'll get back a representation that is the same size as another sequence. This makes downstream ML, especially with other NNs, nice and easy since we don't have to worry about padding.

We can now use `pandarallel` to make multiple requests in parallel. One of the advantages of a GPU-backed REST API is that you can achieve parallelization by using local CPU threads to make multiple REST requests. So let's get the embeddings for *all* the sequences we want to train and test on.

In [20]:
def get_extract_embeddings(seq):
    """Get embeddings from ESM2 via API and extract the final layer of embeddings.
    
    Normally would do this in one function, but for demonstration purposes,
    will build another function for the apply() here."""
    return [s['33'] for s in get_embeddings(seq)][0]

Quickly test this function:

In [21]:
%%time

embeddings = df.iloc[:2, :].seq.apply(get_extract_embeddings).to_list()

CPU times: user 108 ms, sys: 24.2 ms, total: 132 ms
Wall time: 5.37 s


In [22]:
pd.DataFrame(embeddings)

,0,1,2,3,4,5,6,7,8,9,...,1270,1271,1272,1273,1274,1275,1276,1277,1278,1279
0,-0.002958,0.004187,-0.045949,0.005272,-0.099290,0.002290,-0.037235,0.025133,0.073217,-0.090025,...,0.131799,0.063513,0.020858,0.100373,-0.018374,0.128063,0.079383,-0.006516,-0.016403,-0.021598
1,-0.008707,-0.004934,-0.044122,-0.009039,-0.099484,-0.028994,-0.045413,0.010935,0.075985,-0.089158,...,0.171023,0.060234,-0.015955,0.087368,-0.010338,0.086776,0.096674,0.036780,-0.017618,-0.010025


Now that it works, run it on a larger set of `1k` sequences. When BioLM receives these multiple requests in parallel, it will begin scaling up and loading the model on multiple GPUs for parallelization.

In [13]:
%%time

embeddings_1 = df.iloc[0:500, :].seq.parallel_apply(get_extract_embeddings).to_list()

CPU times: user 855 ms, sys: 364 ms, total: 1.22 s
Wall time: 6min 54s


Multiple servers should be hot now, so let's see the time to get the next set of `1k` embeddings. It should be significantly faster.

In [ ]:
%%time

embeddings_2 = df.iloc[500:1000, :].seq.parallel_apply(get_extract_embeddings).to_list()

With that demonstration done, let's do the remainder.

In [ ]:
%%time

embeddings_3 = df.iloc[1000:, :].seq.parallel_apply(get_extract_embeddings).to_list()

In [ ]:
embeddings = pd.concat([
    pd.DataFrame(embeddings_1),
    pd.DataFrame(embeddings_2),
    pd.DataFrame(embeddings_3),
], axis=0)

In [ ]:
embeddings.sample(3)

## Modeling with XGBoost

In order to not require any transformations of the data, let's use a tree-based method to quickly create a regression model using these ESM2 embeddings.

In [ ]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(
    embeddings,
    df.label,
    test_size=0.2,
    random_state=54
)

print("X Train size: {}\nX Test size: {}".format(train_x.shape, test_x.shape))
print("Y Train size: {}\nY Test size: {}".format(train_y.shape, test_y.shape))

In [ ]:
#Set up cross-validation modeling objective
data_dmatrix = xgb.DMatrix(data=train_x, label=train_y)

params = {
    "objective": "reg:linear",
    'colsample_bytree': 0.40,
    'learning_rate': 0.1,
    'max_depth': 100,
    'alpha': 10,
}

# Run CV
cv_results = xgb.cv(
    dtrain=data_dmatrix,
    params=params,
    nfold=3,
    num_boost_round=100,
    early_stopping_rounds=10,
    metrics="rmse",
    as_pandas=True,
    seed=123
)

We can see where the performance started...

In [ ]:
cv_results.head()

But the best results are at the tail end, since this model attempts to converge upon a good fit.

In [ ]:
cv_results.tail(10)

In [ ]:
# Final RMSE on test set
print((cv_results["test-rmse-mean"]).tail(1))

In [ ]:
# Final SD on test set
print((cv_results["test-rmse-std"]).tail(1))

We can get context for these values by looking at the `Y` values that were used in this cross-validation experiment.

In [ ]:
plt.figure(figsize=(6, 5))

train_y.hist()

Now we can train a model with the cross-validated parameters, using our full training dataset.

In [ ]:
xg_reg = xgb.train(params=params, dtrain=data_dmatrix, num_boost_round=80)

Measuring the predicted values against our test set, let's see how well the model did using just the sequence embeddings features.

In [ ]:
y_pred = xg_reg.predict(xgb.DMatrix(data=test_x, label=test_y))

In [ ]:
sns.regplot(x=test_y, y=y_pred)

Lastly, we can attempt to inspect the model and learn a bit more about it, its fit, and our data.

In [ ]:
xgb.plot_importance(xg_reg, max_num_features=20, grid=False)
plt.rcParams['figure.figsize'] = [10, 8]
plt.show()

In [ ]:
xgb.plot_tree(xg_reg,num_trees=0)
plt.rcParams['figure.figsize'] = [50, 10]
plt.show()